In [1]:
import unicodecsv
import django
import codecs
import json
django.setup()
from sefaria.model import *
from collections import defaultdict

# manual review
# Aspaklaria topics without WikiData matches: Fine, can try to add edges for more important topics
# Aspaklaria topics with more than one match: Manually disambiguate
# Wikidata nodes without Aspaklaria: Should add as topics
# Wikidata nodes without Hebrew: Should fill in Hebrew name
# Wikidata edges: make sure they make sense

WARNING Failed to load 're2'.  Falling back to 're' for regular expression parsing. See https://github.com/Sefaria/Sefaria-Project/wiki/Regular-Expression-Engines
/Users/nss/.pyenv/versions/2.7.13/lib/python2.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
WARNING Failed to load 're2'.  Falling back to 're' for regular expression parsing. See https://github.com/blockspeiser/Sefaria-Project/wiki/Regular-Expression-Engines


In [2]:
with codecs.open("wikidata_people_final.json", "rb", encoding='utf8') as fin:
    wiki = json.load(fin)

In [25]:
wiki
# wiki[id][aliases][en|he][0][value]

{u'Q10271944': {u'aliases': {u'en': [{u'language': u'en',
     u'value': u'Elishama, son of Ammihud'},
    {u'language': u'en', u'value': u'Elishama ben Ammihud'}]},
  u'heName': u'\u05d0\u05dc\u05d9\u05e9\u05de\u05e2 \u05d1\u05df \u05e2\u05de\u05d9\u05d4\u05d5\u05d3',
  u'name': u'Elishama the son of Ammihud',
  u'position held': [{u'id': u'Q1251441', u'string': u'leader'}],
  u'present in work': [{u'id': u'Q43099', u'string': u'Numbers'},
   {u'id': u'Q9813916', u'string': u'1 Chronicles'}],
  u'presentIn': [u'Numbers', u'I Chronicles'],
  u'sex or gender': [{u'id': u'Q6581097', u'string': u'male'}]},
 u'Q10287821': {u'aliases': {},
  u'father': [{u'id': u'Q25768260', u'string': u'Ebed'}],
  u'heName': u'\u05d2\u05e2\u05dc \u05d1\u05df \u05e2\u05d1\u05d3',
  u'name': u'Gaal',
  u'present in work': [{u'id': u'Q81240', u'string': u'Judges'}],
  u'presentIn': [u'Judges'],
  u'sex or gender': [{u'id': u'Q6581097', u'string': u'male'}]},
 u'Q10297002': {u'aliases': {u'en': [{u'language': 

In [5]:
with open("aspaklaria_people.csv", "rb") as fin:
    csv = unicodecsv.DictReader(fin)
    asp = []
    for row in csv:
        asp += [row]
    asp_bib = filter(lambda x: x['category1'] == 'biblical person' or x['category2'] == 'biblical person', asp)
    asp_tal = filter(lambda x: x['category1'] == 'talmudic person' or x['category2'] == 'talmudic person', asp)
    asp_mis = filter(lambda x: x['category1'] == 'mishnaic person' or x['category2'] == 'mishnaic person', asp)

In [6]:
wiki_full_name = defaultdict(list)
wiki_first_name = defaultdict(list)
wiki_alias = defaultdict(list)
wiki_all = []
for _id, wiki_guy in wiki.items():
    if len(wiki_guy.get('heName','')) > 0:
        item = {'id': _id, 'name': wiki_guy['name'], 'heName': wiki_guy['heName'], 'matched': False}
        wiki_all += [item]
        wiki_full_name[wiki_guy['heName']] += [item]
        first_name = wiki_guy['heName'].split()[0]
        wiki_first_name[first_name] += [item]
        for alias in wiki_guy['aliases'].get('he', []):
            wiki_alias[alias['value']] += [item]
asp_map = {}
for asp_guy in asp_bib:
    full_name = asp_guy['he']
    first_name = full_name.split()[0]
    full_name_match = wiki_full_name.get(full_name, [])
    first_name_match = wiki_first_name.get(first_name, [])
    alias_match = wiki_alias.get(full_name, [])
    if first_name != full_name:
        alias_match += wiki_alias.get(first_name, [])
    for yo in full_name_match:
        yo['matched'] = True
    for yo in first_name_match:
        yo['matched'] = True
    for yo in alias_match:
        yo['matched'] = True
    asp_map[asp_guy['he']] = {'full_name': full_name_match, 'first_name': first_name_match, 'alias': alias_match}

In [8]:
count = 0
ambig = 0
total = 0
for asp_guy, val in asp_map.items():
    if len(val['first_name']) == 1 or len(val['full_name']) == 1 or len(val['alias']) == 1:
        count +=1
    elif len(val['first_name']) > 1 or len(val['full_name']) > 1 or len(val['alias']) > 1:
        ambig += 1
    else:
        print asp_guy, val
    total += 1

בארה {'alias': [], 'first_name': [], 'full_name': []}
ציבא {'alias': [], 'first_name': [], 'full_name': []}
בלדד {'alias': [], 'first_name': [], 'full_name': []}
חזאל {'alias': [], 'first_name': [], 'full_name': []}
נתנאל {'alias': [], 'first_name': [], 'full_name': []}
שלומיאל {'alias': [], 'first_name': [], 'full_name': []}
חבקוק {'alias': [], 'first_name': [], 'full_name': []}
אמתלאה {'alias': [], 'first_name': [], 'full_name': []}
שלום בן תקוה {'alias': [], 'first_name': [], 'full_name': []}
שונמית {'alias': [], 'first_name': [], 'full_name': []}
נחמיה {'alias': [], 'first_name': [], 'full_name': []}
ידותון {'alias': [], 'first_name': [], 'full_name': []}
מיכה הנביא {'alias': [], 'first_name': [], 'full_name': []}
עמוס {'alias': [], 'first_name': [], 'full_name': []}
עידית {'alias': [], 'first_name': [], 'full_name': []}
מישאל ואלצפן {'alias': [], 'first_name': [], 'full_name': []}
למואל {'alias': [], 'first_name': [], 'full_name': []}
ממרא {'alias': [], 'first_name': [], 'full_nam

In [61]:
count, total, ambig

(228, 355, 9)

In [10]:
# Aspaklaria matched / unmatched

rows = []
max_match = -1
for name, val in asp_map.items():
    item = {'Name': name}
    matches = val['full_name'] + val['first_name'] + val['alias']
    ids_seen = set()
    for i, m in enumerate(matches):
        if m['id'] in ids_seen:
            continue
        ids_seen.add(m['id'])
        item['Match Name {}'.format(i+1)] = m['heName']
        item['Match URL {}'.format(i+1)] = 'https://www.wikidata.org/wiki/' + m['id']
        if i > max_match:
            max_match = i
    rows += [item]
with open('people_tanakh_matched.csv', 'wb') as fout:
    csv = unicodecsv.DictWriter(fout, ['Name'] + map(lambda x: 'Match Name {}'.format(x+1), xrange(max_match+1)) + map(lambda x: 'Match URL {}'.format(x+1), xrange(max_match+1)))
    csv.writeheader()
    csv.writerows(rows)

In [11]:
# WikiData unmatched

rows = []
for yo in wiki_all:
    if not yo['matched']:
        rows += [{'Hebrew Name': yo['heName'], 'English Name': yo['name'], 'URL': 'https://www.wikidata.org/wiki/' + yo['id']}]
with open('people_tanakh_unmatched.csv', 'wb') as fout:
    csv = unicodecsv.DictWriter(fout, ['Hebrew Name', 'English Name', 'URL'])
    csv.writeheader()
    csv.writerows(rows)

In [4]:
# WikiData edges
check_id = {'father', 'mother', 'child', 'spouse'}
edge_map = {
    'father': u'child of',
    'mother': u'child of',
    'sex or gender': u'has gender',
    'child': u'parent of',
    'aliases': u'alternate spelling of',
    'spouse': u'spouse of',
    'position held': u'has role',
    'occupation': u'has role',
}
occ_map = {
    'sovereign': u'מלך מלכות',
    'monarch': u'מלך מלכות',
    'regent': u'מלך מלכות',
    'king': u'מלך מלכות',
    'Kohen': [u'has specific dependence', u'כהונה'],
    'priest': [u'has specific dependence', u'כהונה'],
    'military leader': u'מנהיג',
    'religious leader': u'מנהיג',
    'statesperson': u'מנהיג',
    'midwife': u'מילדת',
    'maid': u'שפחה',
    'consort': u'שפחה',
    'shepherd': u'רועה רעה',
    'herder': u'רועה רעה',
    'High Priest of Israel': u'כהן גדול',
    'high priest': u'כהן גדול',
    'judge': u'שופט',
    'prophet': u'נביא',
    'Nazirite': u'נזיר',
}
pos_held_map = {
    'biblical judge': u'שופט',
    'judge': u'שופט',
    'King of Israel': u'מלך ישראל',
    'King of Judah': u'מלך יהודה',
    'High Priest of Israel': u'כהן גדול'
}
for _id, wiki_guy in wiki.items():
    wiki_guy['edges'] = {}
    for e, f in edge_map.items():
        if e not in edge_map:
            continue
        val_list = wiki_guy.get(e, [])
        if e == 'aliases':
            val_list = wiki_guy.get(e, {}).get('he', []) + wiki_guy.get(e, {}).get('en', [])
        for val in val_list:
            final_val = val.get('string', val.get('value', None))
            final_edge = f
            final_id = None
            if e in check_id:
                if val['id'] not in wiki:
                    continue
                final_id = val['id']
            if final_val is None:
                print 'OH NO@'
            if e == 'aliases':
                if val.get('language', '') == 'en':
                    final_edge = 'has transliteration'
                final_val = val['value']
            elif e == 'position held':
                if final_val not in pos_held_map:
                    continue
                final_val = pos_held_map[final_val]
            elif e == 'occupation':
                if final_val not in occ_map:
                    continue
                final_val = occ_map[final_val]
                if isinstance(final_val, list):
                    final_edge = final_val[0]
                    final_val = final_val[1]
            #print final_edge, final_val
            wiki_guy['edges'][u'{}|{}'.format(final_edge, final_val)] = {'edge':  final_edge, 'value':  final_val, 'valueID': final_id}
rows = []
for _id, wiki_guy in wiki.items():
    for _, edge in wiki_guy['edges'].items():
        #print edge['edge'], edge['value']

        rows += [{'ID': _id, 'Name': wiki_guy.get('name', ''), 'He Name': wiki_guy.get('heName', ''), 'Edge': edge['edge'], 'Value': edge['value'], 'Value ID': edge.get('valueID', '')}]
with open('people_tanakh_edges.csv', 'wb') as fout:
    print 'yoyo'
    csv = unicodecsv.DictWriter(fout, ['ID', 'Name', 'He Name', 'Edge', 'Value', 'Value ID'])
    csv.writeheader()
    csv.writerows(rows)

yoyo


{}
{u'alternate spelling of|\u05d0\u05d3\u05d5\u05e0\u05d9\u05d4 \u05d1\u05df \u05d7\u05d2\u05d9\u05ea': {'edge': u'alternate spelling of', 'value': u'\u05d0\u05d3\u05d5\u05e0\u05d9\u05d4 \u05d1\u05df \u05d7\u05d2\u05d9\u05ea'}, u'alternate spelling of|\u05d0\u05d3\u05e0\u05d9\u05d4': {'edge': u'alternate spelling of', 'value': u'\u05d0\u05d3\u05e0\u05d9\u05d4'}, u'alternate spelling of|\u05d0\u05d3\u05d5\u05e0\u05d9\u05d4\u05d5': {'edge': u'alternate spelling of', 'value': u'\u05d0\u05d3\u05d5\u05e0\u05d9\u05d4\u05d5'}}
{}
{}
{u'has transliteration|Jered': {'edge': 'has transliteration', 'value': u'Jered'}}
{}
{}
{}
{}
{u'alternate spelling of|\u05d0\u05b7\u05d7\u05b0\u05d0\u05b8\u05d1': {'edge': u'alternate spelling of', 'value': u'\u05d0\u05b7\u05d7\u05b0\u05d0\u05b8\u05d1'}}
{u'has transliteration|Azaria': {'edge': 'has transliteration', 'value': u'Azaria'}}
{}
{u'has transliteration|Zuleika': {'edge': 'has transliteration', 'value': u'Zuleika'}, u'has transliteration|Zulaika': {'e